In [1]:
import numpy as np
import pandas as pd
import env
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split



In [2]:
query = """
    SELECT p.id, transactiondate, parcelid, bathroomcnt, bedroomcnt, calculatedfinishedsquarefeet as squarefeet,
    fips, latitude,  longitude, regionidcity as city, regionidcounty as county, regionidzip as zipcode, yearbuilt, taxvaluedollarcnt as price FROM predictions_2017 p
JOIN properties_2017 USING (parcelid)
WHERE transactiondate BETWEEN '2017-05-01' AND '2017-06-30'
AND bedroomcnt >0 AND bathroomcnt>0 AND `propertylandusetypeid` = 261
    """

In [3]:
df = pd.read_sql(query, env.get_db_url('zillow'))

In [4]:
df.head()

,id,transactiondate,parcelid,bathroomcnt,bedroomcnt,squarefeet,fips,latitude,longitude,city,county,zipcode,yearbuilt,price
0,1248,2017-06-23,11289917,2.0,3.0,1458.0,6037.0,34686163.0,-118113100.0,5534.0,3101.0,97318.0,1970.0,136104.0
1,1772,2017-06-30,11705026,1.0,2.0,1421.0,6037.0,33999877.0,-118291863.0,12447.0,3101.0,96018.0,1911.0,35606.0
2,2028,2017-06-01,14269464,3.0,4.0,2541.0,6059.0,33694636.0,-117912245.0,38032.0,1286.0,96958.0,2003.0,880456.0
3,3273,2017-06-01,11389003,2.0,3.0,1650.0,6037.0,33985377.0,-118361620.0,396550.0,3101.0,96037.0,1949.0,614000.0
4,3429,2017-06-29,11967869,1.0,2.0,693.0,6037.0,34112946.0,-118208106.0,12447.0,3101.0,96023.0,1921.0,274237.0


In [5]:
df.describe()

,id,parcelid,bathroomcnt,bedroomcnt,squarefeet,fips,latitude,longitude,city,county,zipcode,yearbuilt,price
count,14990.000000,1.499000e+04,14990.000000,14990.000000,14990.000000,14990.000000,1.499000e+04,1.499000e+04,14701.000000,14990.000000,14982.000000,14974.000000,1.499000e+04
mean,42507.143763,1.302688e+07,2.329153,3.316878,1943.132355,6049.381721,3.402249e+07,-1.181957e+08,33076.205088,2515.726484,96651.749499,1963.886470,5.416945e+05
std,6467.351347,3.723348e+06,1.019375,0.936250,1001.275394,21.225607,2.760549e+05,3.593028e+05,47688.437636,806.445112,4970.071943,23.314756,7.309388e+05
min,1248.000000,1.071210e+07,1.000000,1.000000,300.000000,6037.000000,3.334085e+07,-1.193883e+08,3491.000000,1286.000000,95982.000000,1878.000000,1.050400e+04
25%,37038.250000,1.150300e+07,2.000000,3.000000,1276.000000,6037.000000,3.382366e+07,-1.184062e+08,12447.000000,1286.000000,96208.000000,1950.000000,1.990028e+05
50%,42558.500000,1.258458e+07,2.000000,3.000000,1680.000000,6037.000000,3.402708e+07,-1.181547e+08,24812.000000,3101.000000,96414.000000,1961.000000,3.851740e+05
75%,48034.750000,1.414273e+07,3.000000,4.000000,2346.000000,6059.000000,3.419064e+07,-1.179260e+08,40227.000000,3101.000000,96996.000000,1980.000000,6.449650e+05
max,53501.000000,1.676392e+08,11.000000,12.000000,15450.000000,6111.000000,3.477966e+07,-1.175554e+08,396556.000000,3101.000000,399675.000000,2015.000000,2.385837e+07


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14990 entries, 0 to 14989
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               14990 non-null  int64  
 1   transactiondate  14990 non-null  object 
 2   parcelid         14990 non-null  int64  
 3   bathroomcnt      14990 non-null  float64
 4   bedroomcnt       14990 non-null  float64
 5   squarefeet       14990 non-null  float64
 6   fips             14990 non-null  float64
 7   latitude         14990 non-null  float64
 8   longitude        14990 non-null  float64
 9   city             14701 non-null  float64
 10  county           14990 non-null  float64
 11  zipcode          14982 non-null  float64
 12  yearbuilt        14974 non-null  float64
 13  price            14990 non-null  float64
dtypes: float64(11), int64(2), object(1)
memory usage: 1.6+ MB


In [7]:
df = df.drop(columns = 'city')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14990 entries, 0 to 14989
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               14990 non-null  int64  
 1   transactiondate  14990 non-null  object 
 2   parcelid         14990 non-null  int64  
 3   bathroomcnt      14990 non-null  float64
 4   bedroomcnt       14990 non-null  float64
 5   squarefeet       14990 non-null  float64
 6   fips             14990 non-null  float64
 7   latitude         14990 non-null  float64
 8   longitude        14990 non-null  float64
 9   county           14990 non-null  float64
 10  zipcode          14982 non-null  float64
 11  yearbuilt        14974 non-null  float64
 12  price            14990 non-null  float64
dtypes: float64(10), int64(2), object(1)
memory usage: 1.5+ MB


In [9]:
df = df.dropna()

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14967 entries, 0 to 14989
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               14967 non-null  int64  
 1   transactiondate  14967 non-null  object 
 2   parcelid         14967 non-null  int64  
 3   bathroomcnt      14967 non-null  float64
 4   bedroomcnt       14967 non-null  float64
 5   squarefeet       14967 non-null  float64
 6   fips             14967 non-null  float64
 7   latitude         14967 non-null  float64
 8   longitude        14967 non-null  float64
 9   county           14967 non-null  float64
 10  zipcode          14967 non-null  float64
 11  yearbuilt        14967 non-null  float64
 12  price            14967 non-null  float64
dtypes: float64(10), int64(2), object(1)
memory usage: 1.6+ MB


In [11]:
# convert zip code to categorical
df.zipcode = df.zipcode.astype(int).astype(str)

In [12]:
df.zipcode

0        97318
1        96018
2        96958
3        96037
4        96023
         ...  
14985    97018
14986    97067
14987    96523
14988    97065
14989    96129
Name: zipcode, Length: 14967, dtype: object

In [13]:
fip = {
    'fips': [6037, 6059, 6111],
    'County':['Los Angles', 'Orange', 'Ventura']
}

In [15]:
fips = pd.DataFrame(fip)

In [16]:
fips

,fips,County
0,6037,Los Angles
1,6059,Orange
2,6111,Ventura


In [17]:
df = pd.merge(df, fips, left_on = 'fips', right_on = 'fips', how = 'left')

In [18]:
df = df.drop(columns = 'fips_county')

KeyError: "['fips_county'] not found in axis"

In [19]:
df.head(3)

,id,transactiondate,parcelid,bathroomcnt,bedroomcnt,squarefeet,fips,latitude,longitude,county,zipcode,yearbuilt,price,County
0,1248,2017-06-23,11289917,2.0,3.0,1458.0,6037.0,34686163.0,-118113100.0,3101.0,97318,1970.0,136104.0,Los Angles
1,1772,2017-06-30,11705026,1.0,2.0,1421.0,6037.0,33999877.0,-118291863.0,3101.0,96018,1911.0,35606.0,Los Angles
2,2028,2017-06-01,14269464,3.0,4.0,2541.0,6059.0,33694636.0,-117912245.0,1286.0,96958,2003.0,880456.0,Orange


In [20]:
train, test = train_test_split(df, train_size = .70, random_state = 100)

In [24]:
train.head()

,id,transactiondate,parcelid,bathroomcnt,bedroomcnt,squarefeet,fips,latitude,longitude,county,zipcode,yearbuilt,price,County
14113,52239,2017-06-29,11428689,2.0,3.0,1291.0,6037.0,33905093.0,-118319125.0,3101.0,96106,1957.0,268863.0,Los Angles
3456,36635,2017-05-16,13849709,2.0,4.0,1833.0,6059.0,33942120.0,-117970825.0,1286.0,96186,1956.0,476193.0,Orange
9743,45905,2017-06-13,13065441,2.0,4.0,1200.0,6037.0,34060299.0,-117795042.0,3101.0,96508,1954.0,157915.0,Los Angles
1244,33380,2017-05-05,14413214,4.0,5.0,3586.0,6059.0,33517619.0,-117697388.0,1286.0,96987,1994.0,869805.0,Orange
7511,42594,2017-06-01,12869924,1.0,3.0,1222.0,6037.0,34086978.0,-117713653.0,3101.0,96469,1952.0,461938.0,Los Angles


In [31]:
df[df.zipcode == '399675']

,id,transactiondate,parcelid,bathroomcnt,bedroomcnt,squarefeet,fips,latitude,longitude,county,zipcode,yearbuilt,price,County
2952,35872,2017-05-12,12969877,6.0,5.0,4698.0,6037.0,34160173.0,-117973852.0,3101.0,399675,1994.0,4548320.0,Los Angles
4854,38690,2017-05-23,12969536,2.0,3.0,1532.0,6037.0,34144897.0,-117953095.0,3101.0,399675,1954.0,69537.0,Los Angles
7450,42510,2017-06-01,12969457,3.0,2.0,2143.0,6037.0,34154731.0,-117954681.0,3101.0,399675,1981.0,753001.0,Los Angles
11914,49034,2017-06-21,12969466,2.0,3.0,1895.0,6037.0,34153603.0,-117953027.0,3101.0,399675,1962.0,719700.0,Los Angles


In [26]:
train.groupby('zipcode').price.median()

zipcode
399675    753001.0
95982      33084.0
95983     190816.5
95984     148542.0
95985     691076.5
            ...   
97328     147200.0
97329     263500.0
97330     199000.0
97331     194000.0
97344     101401.0
Name: price, Length: 370, dtype: float64

In [39]:
train.drop(train[train.zipcode == '399675'].index, inplace = True)

In [42]:
median_price = pd.Series(train.groupby('zipcode').price.median())

In [43]:
median_price

zipcode
95982     33084.0
95983    190816.5
95984    148542.0
95985    691076.5
95986    528450.0
           ...   
97328    147200.0
97329    263500.0
97330    199000.0
97331    194000.0
97344    101401.0
Name: price, Length: 369, dtype: float64

In [44]:
df['median_price'] = df.zipcode.map(median_price)

In [46]:
df.head(4)

,id,transactiondate,parcelid,bathroomcnt,bedroomcnt,squarefeet,fips,latitude,longitude,county,zipcode,yearbuilt,price,County,median_price
0,1248,2017-06-23,11289917,2.0,3.0,1458.0,6037.0,34686163.0,-118113100.0,3101.0,97318,1970.0,136104.0,Los Angles,149204.0
1,1772,2017-06-30,11705026,1.0,2.0,1421.0,6037.0,33999877.0,-118291863.0,3101.0,96018,1911.0,35606.0,Los Angles,182416.0
2,2028,2017-06-01,14269464,3.0,4.0,2541.0,6059.0,33694636.0,-117912245.0,1286.0,96958,2003.0,880456.0,Orange,489381.0
3,3273,2017-06-01,11389003,2.0,3.0,1650.0,6037.0,33985377.0,-118361620.0,3101.0,96037,1949.0,614000.0,Los Angles,653668.5
